In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive/MyDrive')
os.listdir('src')

Mounted at /content/drive


['models', '__pycache__', 'model_trainer.py', 'transformations.py', 'utils.py']

In [2]:
from src.models.our_model import OurModel
from src.models.pretrained_models import VGG16Pretrained, ResNetPretrained
import torch
from src.models.ensemble import HardVotingEnsemble, SoftVotingEnsemble
from src.utils import load_data, evaluate_model
from src.model_trainer import ModelTrainer
from src.transformations import normalized_simple_transform
device_str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_str)
torch.manual_seed(123)
print(device)
torch.set_num_threads(2)

cuda


In [3]:
# cell for donwloading data

import os
import tarfile
import urllib.request
import shutil

download_dir = "/data"

if not os.path.exists(download_dir):
    os.makedirs(download_dir)

tar_path = os.path.join(download_dir, "CINIC-10.tar.gz")
cinic10_url = "https://datashare.is.ed.ac.uk/bitstream/handle/10283/3192/CINIC-10.tar.gz"
urllib.request.urlretrieve(cinic10_url, tar_path)
print("Finished downloading")

with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall(path=download_dir)

extracted_dir = os.path.join(download_dir, "CINIC-10")

if os.path.exists(extracted_dir) and os.path.isdir(extracted_dir):
    for item in os.listdir(extracted_dir):
        src_path = os.path.join(extracted_dir, item)
        dst_path = os.path.join(download_dir, item)
        shutil.move(src_path, dst_path)

print(download_dir)

Finished downloading
/data


In [4]:
test_loader = load_data('/data/test', batch_size=256, shuffle=True, transform=normalized_simple_transform(), num_workers=1)

## all models ensemble

In [6]:
vgg16 = VGG16Pretrained(device="cpu")
vgg16.load_state_dict(torch.load("./saved_models/vgg_pretrained/horizontal_flip_20/VGG16Pretrained_20.pth", map_location=device))
vgg16.to(device)
ourmodel = OurModel(aux_enabled=False, se_squeeze=8)
ourmodel.load_state_dict(torch.load("./saved_models/ourmodel/combined_20/OurModel_16.pth", map_location=device))
ourmodel.to(device)
resnet = ResNetPretrained()
resnet.load_state_dict(torch.load("./saved_models/resnet/final/ResNetPretrained_20.pth", map_location=device))
resnet.to(device)
base_models = [ourmodel, vgg16, resnet]

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 165MB/s]


In [7]:
hard_vot_ensemble = HardVotingEnsemble(base_models)
hard_vot_ensemble.to(device)

HardVotingEnsemble()

In [8]:
metrics_hard = evaluate_model(hard_vot_ensemble, test_loader, device)
print(f"\n {metrics_hard}")

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 
 {'accuracy': 0.8021777777777778, 'f1_score': 0.8009975569622965, 'roc_auc': None}


In [9]:
soft_vot_ensemble = SoftVotingEnsemble(base_models)
soft_vot_ensemble.to(device)

SoftVotingEnsemble()

In [11]:
metrics_soft = evaluate_model(soft_vot_ensemble, test_loader, device)
print(f"\n {metrics_soft}")

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 
 {'accuracy': 0.8170777777777778, 'f1_score': 0.8157499424139534, 'roc_auc': None}
